In [3]:
# 🔧 Install required packages
!pip install -q transformers sentence-transformers fuzzywuzzy[speedup] faiss-cpu pandas
# Facebook AI Similarity Search (FAISS)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 26.5 MB/s eta 0:00:00


In [4]:
# Imports
import pandas as pd
import faiss
from transformers import pipeline
from sentence_transformers import SentenceTransformer
from fuzzywuzzy import fuzz
import torch
import json



In [5]:
# Step 1: Sample input (can be structured or free-text descriptions)
transactions = [
    {"id": 1, "description": "Transfer to ShellCorp Intl in Cayman Islands."},
    {"id": 2, "description": "Donation sent to GlobalUnity Non-Profit Org."},
    {"id": 3, "description": "Payment made to XBridge Ventures, previously flagged by compliance."},
    {"id": 4, "description": "Wire transfer to Oceanic Holdings in Singapore."},
    {"id": 5, "description": "Contribution received by HelpingHands Charity Foundation."},
    {"id": 6, "description": "Settlement payment processed for MetroFinance Inc, under review."},
    {"id": 7, "description": "Invoice payment to GlobalTrade Partners Ltd in Dubai."},
    {"id": 8, "description": "Refund issued by RetailCentral Corp for overcharge."},
    {"id": 9, "description": "Reimbursement from BlueChip LLC for travel expenses."},
    {"id": 10, "description": "Payment processed to SecureInvestments, pending further verification."}
]


df = pd.DataFrame(transactions)



In [6]:
# Step 2: Load NER model (HF)
ner_pipeline = pipeline("ner", grouped_entities=True)

# Step 3: Named Entity Extraction
df["entities"] = df["description"].apply(lambda x: [ent['word'] for ent in ner_pipeline(x)])



No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision 4c53496 (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/998 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

Device set to use cuda:0
/usr/local/lib/python3.11/dist-packages/transformers/pipelines/token_classification.py:170: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="AggregationStrategy.SIMPLE"` instead.
  warnings.warn(


In [7]:
# Step 4: Create embedding model + index of known safe/flagged entities
known_entities = ["Shell Corporation International", "UNICEF", "World Bank", "XBridge Ventures", "GlobalUnity Foundation"]
embed_model = SentenceTransformer("all-MiniLM-L6-v2")
known_embeds = embed_model.encode(known_entities)

index = faiss.IndexFlatL2(known_embeds.shape[1])
index.add(known_embeds)



In [8]:
# Step 5: Verify and risk score
def verify_and_score(extracted_entities):
    result = []
    risk_keywords = ['shell', 'offshore', 'flagged', 'cayman']

    for entity in extracted_entities:
        embed = embed_model.encode([entity])
        _, I = index.search(embed, k=1)
        match = known_entities[I[0][0]]
        fuzz_score = fuzz.partial_ratio(entity.lower(), match.lower())

        # Heuristic risk scoring
        keyword_risk = any(risk in entity.lower() for risk in risk_keywords)
        risk = 70 if keyword_risk else 30
        if fuzz_score < 60:
            risk += 20

        result.append({
            "entity": entity,
            "matched_to": match,
            "match_score": fuzz_score,
            "risk_score": min(risk, 100)
        })
    return result

df["risk_eval"] = df["entities"].apply(verify_and_score)



In [9]:
# Step 6: Generate LLM-based justifications using FLAN-T5
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

llm = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(llm)
model = AutoModelForSeq2SeqLM.from_pretrained(llm)



tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [11]:
def justify(entity, match, score):
    prompt = f"""
    You are an expert risk analysis assistant.

    Analyze the entity: "{entity}"
    It matched to: "{match}"
    It has been assigned a risk score of {score}.

    Briefly explain why this entity might carry that level of risk.
    """

    inputs = tokenizer(prompt.strip(), return_tensors="pt", truncation=True).to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=80,
        do_sample=False,  # deterministic output
        early_stopping=True
    )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)


# Apply justification per entity
justifications = []
for row in df.itertuples():
    reasons = []
    for item in row.risk_eval:
        reason = justify(item['entity'], item['matched_to'], item['risk_score'])
        item['justification'] = reason
        reasons.append(item)
    justifications.append(reasons)

df["risk_eval"] = justifications

# Step 7: Output final result
print("Final Risk Analysis Output (JSON-like):\n")
for row in df.itertuples():
    output = {
        "transaction_id": row.id,
        "description": row.description,
        "entities": row.risk_eval
    }
    print(json.dumps(output, indent=2))


Final Risk Analysis Output (JSON-like):

{
  "transaction_id": 1,
  "description": "Transfer to ShellCorp Intl in Cayman Islands.",
  "entities": [
    {
      "entity": "ShellCorp Intl",
      "matched_to": "Shell Corporation International",
      "match_score": 71,
      "risk_score": 70,
      "justification": "ShellCorp Intl is a subsidiary of Shell Corporation International."
    },
    {
      "entity": "Cayman Islands",
      "matched_to": "World Bank",
      "match_score": 30,
      "risk_score": 90,
      "justification": "The Cayman Islands is a member of the World Bank."
    }
  ]
}
{
  "transaction_id": 2,
  "description": "Donation sent to GlobalUnity Non-Profit Org.",
  "entities": [
    {
      "entity": "GlobalUnity Non",
      "matched_to": "GlobalUnity Foundation",
      "match_score": 87,
      "risk_score": 30,
      "justification": "Global Unity Non is a non-profit."
    },
    {
      "entity": "Org",
      "matched_to": "UNICEF",
      "match_score": 0,
      "r

In [ ]:
# 🔗 OpenCorporates API lookup function
# This has been commented as API token request was rejected.
# import requests

# def lookup_entity_opencorp(name, jurisdiction_code=""):
#     api_key = "YOUR_API_KEY"  #Replace this with your actual API token
#     base_url = "https://api.opencorporates.com/v0.4/companies/search"

#     params = {
#         "q": name,
#         "api_token": api_key
#     }
#     if jurisdiction_code:
#         params["jurisdiction_code"] = jurisdiction_code

#     try:
#         response = requests.get(base_url, params=params)
#         data = response.json()
#         if "results" in data and data["results"]["companies"]:
#             top_result = data["results"]["companies"][0]["company"]
#             return {
#                 "name": top_result.get("name"),
#                 "jurisdiction": top_result.get("jurisdiction_code"),
#                 "company_number": top_result.get("company_number"),
#                 "status": top_result.get("current_status"),
#                 "source_url": top_result.get("opencorporates_url")
#             }
#         else:
#             return {"error": "No match found"}
#     except Exception as e:
#         return {"error": str(e)}


In [ ]:
#  Phase 2 integration: Add LLM justification + OpenCorporates verification
# justifications = []
# for row in df.itertuples():
#     enriched_entities = []
#     for item in row.risk_eval:
#         # LLM-based justification
#         reason = justify(item['entity'], item['matched_to'], item['risk_score'])
#         item['justification'] = reason

#         #  Real-time entity verification using OpenCorporates
#         api_result = lookup_entity_opencorp(item['entity'])
#         item['open_corporates'] = api_result  # Attach API enrichment

#         enriched_entities.append(item)
#     justifications.append(enriched_entities)

# # Update DataFrame with enriched info
# df["risk_eval"] = justifications

# # Final JSON-like output
# import json
# print("Enriched Risk Analysis Output with OpenCorporates:\n")
# for row in df.itertuples():
#     output = {
#         "transaction_id": row.id,
#         "description": row.description,
#         "entities": row.risk_eval
#     }
#     print(json.dumps(output, indent=2))
